In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_msgpack('preprocess/IMDB/Split_Notes_Texts_combined.msg')

In [34]:
df.head()

,id,sentiment,review,review_cleaned,exp_split,cseq
0,5814_8,1,With all this stuff going down at the moment w...,with all this stuff going down at the moment w...,train,with all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...","\the classic war of the worlds\ "" by timothy h...",train,\the classic war of the worlds\ by timothy hin...
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,the film starts with a manager ( nicholas bell...,train,the film starts with a manager nicholas bell g...
3,3630_4,0,It must be assumed that those who praised this...,it must be assumed that those who praised this...,test,it must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy and wondrously unpretentious q...,test,superbly trashy and wondrously unpretentious q...


In [3]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
df['cseq'] = df['review_cleaned']

In [6]:
import re
from tqdm import tqdm_notebook

pattern = re.compile(r'\s\W+\s')
texts = list(df['cseq'])
for t in tqdm_notebook(range(len(texts))) :
    texts[t] = re.sub(r'\s+', ' ', pattern.sub(' ', texts[t]))

In [8]:
df['cseq'] = texts

In [9]:
tokenizer = lambda s : s.split(' ')
vocab = CountVectorizer(max_features=10000, tokenizer=tokenizer, stop_words='english')
vocab.fit(list(df[df['exp_split'] == 'train']['cseq']))

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function <lambda> at 0x7f130d2eb0d0>, vocabulary=None)

In [16]:
bow = {}
for key in ['train', 'test'] :
    bow[key] = vocab.transform(list(df[df['exp_split'] == key]['cseq']))

In [17]:
label = {}
for key in ['train', 'test'] :
    label[key] = list(df[df['exp_split'] == key]['sentiment'])

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [19]:
lr = LogisticRegression(class_weight='balanced')
lr.fit(bow['train'].toarray(), label['train'])
predict = lr.predict_proba(bow['test'].toarray())
print(roc_auc_score(label['test'], predict[:, 1]))

/home/jainsarthak/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9269851200000001


In [28]:
from sklearn.preprocessing import normalize
normalised_bow = {}
for key in ['train', 'test'] :
    normalised_bow[key] = normalize(bow[key], norm='l2', copy=True)

In [29]:
lr_norm = LogisticRegression(class_weight='balanced')
lr_norm.fit(normalised_bow['train'].toarray(), label['train'])
predict = lr_norm.predict_proba(normalised_bow['test'].toarray())
print(roc_auc_score(label['test'], predict[:, 1]))

/home/jainsarthak/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.94015584


In [30]:
binary_bow = {}
for key in ['train', 'test'] :
    binary_bow[key] = np.clip(bow[key].toarray(), 0, 1)

In [31]:
lr_bin = LogisticRegression(class_weight='balanced')
lr_bin.fit(binary_bow['train'], label['train'])
predict = lr_bin.predict_proba(binary_bow['test'])
print(roc_auc_score(label['test'], predict[:, 1]))

/home/jainsarthak/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9308907200000001


In [32]:
bin_normalised_bow = {}
for key in ['train', 'test'] :
    bin_normalised_bow[key] = normalize(binary_bow[key], norm='l2', copy=True)

In [33]:
lr_bin = LogisticRegression(class_weight='balanced')
lr_bin.fit(bin_normalised_bow['train'], label['train'])
predict = lr_bin.predict_proba(bin_normalised_bow['test'])
print(roc_auc_score(label['test'], predict[:, 1]))

/home/jainsarthak/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.94310768
